In [4]:
import os
import subprocess
from Bio import SeqIO
from Bio.Blast import NCBIXML

# Define paths to input and output files
human_fa ="human.fa"
mouse_fa = "mouse.fa"
output_file = "blast_results.txt"

# Database name (without extensions)
db_name = r"mouse_db"

# Step 1: Create BLAST database if not already done
#if not all(os.path.exists(f"{db_name}.{ext}") for ext in ["phr", "pin", "psq"]):
#    print("Creating BLAST database...")
#    makeblastdb_cmd = f'makeblastdb -in "{mouse_fa}" -dbtype prot -out "{db_name}"'
#    subprocess.run(makeblastdb_cmd, shell=True, check=True)
#else:
#    print("BLAST database found.")

# Function to perform BLAST search for each human sequence
def run_blast_search(query_seq, query_id, db_name=db_name, e_value_thresh=1e-5):
    # Write the human sequence to a temporary file
    with open("temp_query.fasta", "w") as temp_file:
        temp_file.write(f">{query_id}\n{str(query_seq.seq)}\n")

    # Run BLASTp against the mouse database and save results in XML format
    result_file = "temp_result.xml"
    blast_command = [
        "blastp",
        "-query", "temp_query.fasta",
        "-db", db_name,
        "-out", result_file,
        "-outfmt", "5",
        "-evalue", str(e_value_thresh)
    ]

    try:
        subprocess.run(blast_command, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error running BLAST: {e}")
        return None

    # Parse the BLAST results
    best_hit = None
    with open(result_file) as result_handle:
        blast_records = NCBIXML.read(result_handle)
        if blast_records.alignments:
            # Get the top alignment (best hit)
            best_alignment = blast_records.alignments[0]
            hsp = best_alignment.hsps[0]

            best_hit = {
                "human_id": query_id,
                "mouse_id": best_alignment.hit_def.split(" ")[0],
                "alignment": hsp.sbjct,
                "e_value": hsp.expect,
                "bitscore": hsp.bits
            }

    # Remove temporary files
    os.remove("temp_query.fasta")
    os.remove("temp_result.xml")
    
    return best_hit

# Open the output file for writing results
with open(output_file, "w") as out_handle:
    # Iterate through each sequence in human.fa
    for record in SeqIO.parse(human_fa, "fasta"):
        query_id = record.id  # Get human sequence ID
        result = run_blast_search(record, query_id)

        # If a best hit is found, write it to the output file
        if result:
            out_handle.write(
                f"Human ID: {result['human_id']}\n"
                f"Mouse ID: {result['mouse_id']}\n"
                f"Alignment:\n{result['alignment']}\n"
                f"E-value: {result['e_value']}\n"
                f"Bitscore: {result['bitscore']}\n\n"
            )
